In [50]:
import math
import random
import copy

from itertools import combinations
from itertools import permutations
import numpy as np

In [51]:
class individual:
    def __init__(self,code,startArray):
        self.codeWeight=1
        self.breakPointsWeight=10
        self.code=code
        self.array = []
        self.startArray = startArray
        self.applyReversals()
        self.breakpoints=self.getBreakpoints()
        self.fitness=self.calculateFitnes()
        
    def applyReversals(self):
        self.array = copy.deepcopy(self.startArray)
        for k in range(0,len(self.code),2):
            i = self.code[k]
            j = self.code[k+1]
            for l in range(int((j-i+1)/2)):
                self.array[i+l],self.array[j-l] = self.array[j-l],self.array[i+l]

    def getBreakpoints(self):
        breakPoints=0
        for i in range(1,len(self.array)):
            if abs(self.array[i-1]-self.array[i])>1:
                breakPoints+=1
        return breakPoints
    def calculateFitnes(self):
        return self.breakPointsWeight*self.breakpoints+self.codeWeight*len(self.code)

In [52]:
class MSR:
    def __init__(self,array):
        self.populationSize=math.floor(len(array)*math.log(len(array))) #Velicina populacije je floor(n*log n)
        self.array=array
        self.breakPoints=self.getBreakpoints()
        self.population=[]
        self.initializePopulation()
        self.tournamentSize=math.ceil(0.15*self.populationSize)
        self.bestUnit = copy.deepcopy(self.population[0])
        self.mutationRate = 0.4
        self.numOfIters = 5000
        self.populationSelectionPercentege=0.2
      
    def getBreakpoints(self):
        breakPoints=0
        for i in range(1,len(self.array)-1):
            if abs(self.array[i-1]-self.array[i])>1:
                breakPoints+=1
        return breakPoints
    def generateIndividual(self,size):
        code=[]
        for i in range(size):
            m=math.ceil(random.randrange(1,math.ceil(len(self.array)/10)+1))
            n=math.ceil(random.randrange(0,math.ceil(9*len(self.array)/10-1)))
            begin=n
            end=m+n
            code.append(begin)
            code.append(end)
        return individual(code,self.array)
            
    def initializePopulation(self):
        lowerBound=self.breakPoints
        upperBound=3*len(self.array)
        for i in range(self.populationSize):
            size=random.randrange(lowerBound,upperBound+1)
            self.population.append(self.generateIndividual(size))
    def selection(self,population):
        #print(len(population))
        selected=random.sample(population,max(round(len(population)*(self.tournamentSize/self.populationSize)),1))
#         print(len(population))
#         print(str(self.tournamentSize/self.populationSize)+" "+str(len(population)))
        best=min(selected,key=lambda x:x.fitness)
        #print("BEST: "+str(best.code))
        return best
    def crossover(self,parent1,parent2):
        if len(parent1)==len(parent2):
            while(True):
                firstPoint=random.randrange(0,len(parent1))
                secondPoint=random.randrange(0,len(parent1))
                if firstPoint!=secondPoint:
                    break
            if firstPoint>secondPoint:
                firstPoint,secondPoint=secondPoint,firstPoint
            child1=parent1[:firstPoint]+parent2[firstPoint:secondPoint]+parent1[secondPoint:]
            child2=parent2[:firstPoint]+parent1[firstPoint:secondPoint]+parent2[secondPoint:]
        else:
            if len(parent1)<len(parent2):
                parent1,parent2=parent2,parent1
            point=random.randrange(0,len(parent1)-len(parent2)+1)
            child1=parent1[:point]+parent2+parent1[point+len(parent2):]
            child2=parent1[point:point+len(parent2)]
        return individual(child1,self.array),individual(child2,self.array)

    def mutation1(self,unit):
        while(True):
            i = random.randrange(0,int(len(unit.code)/2))
            j = random.randrange(0,int(len(unit.code)/2))
            if i !=j: #and unit.code[i]!=unit.code[j]:
                break
        if j < i :
            i,j = j,i
        i = 2*i
        j = 2*j
        for k in range(j,len(unit.code)-1):
            unit.code[k] = unit.code[k+1]
        del unit.code[-1]
        for k in range(i,len(unit.code)-1):
            unit.code[k] = unit.code[k+1]
        del unit.code[-1]

        unit.applyReversals()
        unit.breakpoints = unit.getBreakpoints()
        unit.fitness = unit.calculateFitnes()
    def mutation2(self,unit):
        while(True):
            i = random.randrange(0,int(len(unit.code)/2))
            j = random.randrange(0,int(len(unit.code)/2))
            if i !=j:
                break
        if j < i :
            i,j = j,i
        i = 2*i
        j = 2*j
        while(True):
            value1=random.randrange(0,len(unit.array))
            value2=random.randrange(0,len(unit.array))
            if value1!=value2:
                break
        unit.code.insert(i,value1)
        unit.code.insert(j,value2)
        unit.applyReversals()
        unit.breakpoints = unit.getBreakpoints()
        unit.fitness = unit.calculateFitnes()

    def mutation3(self,unit): #jedna pozicija mozda ce trebati 2
        i = random.randrange(0,len(unit.code))
        unit.code[i] = random.randrange(0,len(unit.array))
        unit.applyReversals()
        unit.breakpoints = unit.getBreakpoints()
        unit.fitness = unit.calculateFitnes()
    def mutation4(self,unit): #jedna pozicija mozda ce trebati 2
        [i,j]=random.sample(range(len(unit.code)),2)
        [val1,val2]=random.sample(range(len(unit.array)),2)
        unit.code[i] = val1
        unit.code[j] = val2
        unit.applyReversals()
        unit.breakpoints = unit.getBreakpoints()
        unit.fitness = unit.calculateFitnes()
    def mutate(self,unit):
        choice=0
        if random.random()< self.mutationRate:
            choice=random.randrange(1,5)
        if choice==1 and len(unit.code)>2:
                self.mutation1(unit)
        if choice==2 and len(unit.code)>2:
                self.mutation2(unit)
        if choice==3:
                self.mutation3(unit)
        if choice==4:
                self.mutation4(unit)
    '''
    def generatePopulation(self):
        population = []
        for i in range(0,self.populationSize,2):
            parent1 = self.selection()
            parent2 = self.selection()
            child1,child2 = self.crossover(parent1.code,parent2.code)
            temp=[child1,child2,parent1,parent2]
            sorted(temp,key=lambda x:x.fitness)
            child1=copy.deepcopy(temp[0])
            child2=copy.deepcopy(temp[1])
            if random.random() < self.mutationRate and len(child1.code)>2:
                self.mutation1(child1)
            if random.random() < self.mutationRate and len(child2.code)>2:
                self.mutation1(child2)
            if random.random() < self.mutationRate and len(child1.code)>2:
                self.mutation2(child1)
            if random.random() < self.mutationRate and len(child2.code)>2:
                self.mutation2(child2)
            if random.random() < self.mutationRate:
                self.mutation3(child1)
            if random.random() < self.mutationRate:
                self.mutation3(child2)
            population.append(child1)
            population.append(child2)
        self.population = copy.deepcopy(population)
    '''
    def generatePopulation(self):
        population=[]
        sorted(self.population,key=lambda x: x.fitness)
        sizeGroup1=2*(math.floor(self.populationSelectionPercentege*self.populationSize)//2)
        group1=self.population[:sizeGroup1]
        group2=self.population[sizeGroup1:]
        for i in range(0,int(sizeGroup1),2):
            parent1 = self.selection(group1)
            parent2 = self.selection(group1)
            child1,child2 = self.crossover(parent1.code,parent2.code)
            temp=[child1,child2,parent1,parent2]
            sorted(temp,key=lambda x:x.fitness)
            child1=copy.deepcopy(temp[0])
            child2=copy.deepcopy(temp[1])
            self.mutate(child1)
            self.mutate(child2)
            population.append(child1)
            population.append(child2)
        while len(population)<self.populationSize:
            parent1 = self.selection(group2)
            parent2 = self.selection(group2)
            child1,child2 = self.crossover(parent1.code,parent2.code)
            temp=[child1,child2,parent1,parent2]
            sorted(temp,key=lambda x:x.fitness)
            child1=copy.deepcopy(temp[0])
            child2=copy.deepcopy(temp[1])
            self.mutate(child1)
            self.mutate(child2)
            population.append(child1)
            population.append(child2)
        self.population=copy.deepcopy(population)
            
            
        
    def findBest(self):
        best=min(self.population,key=lambda x:x.fitness)
        return copy.deepcopy(best)

    def solve(self):
        fitnessRepetition=0
        oldFitness=0
        self.initializePopulation()
        for i in range(self.numOfIters):
            best = self.findBest()
            if best.fitness < self.bestUnit.fitness:
                self.bestUnit = copy.deepcopy(best)
            if i%1000 == 0:
                #copyBest=copy.deepcopy(self.bestUnit)
                #copyBest.applyReversals()
                print("Najbolja jedinka:\nFitness: " + str(self.bestUnit.fitness) + "\nNiz: " + str(self.bestUnit.array)+"\nCode: "+str(self.bestUnit.code))
                print()
            if oldFitness==best.fitness:
                fitnessRepetition+=1
            oldFitness=best.fitness
            self.generatePopulation()
            
            #should_break = True
            #for i in range(1,len(self.population)):
            #    if (len(self.population[i-1].code) != len(self.population[i].code)) or (self.population[i-1].fitness != self.population[i].fitness):
            #        should_break = False
            #if should_break:
            #    break
        return self.bestUnit


In [53]:
#msr=MSR([3,2,1,4,6,5,7,9,8])
#msr=MSR([5,1,3,6,4,2,7])
#msr=MSR([10,5,1,3,8,11,6,4,12,2,7,9])
#t=random.sample(range(1,21),20)
#print(t)
#msr=MSR(t)
#msr.solve()
#t=msr.bestUnit
#t=individual([5, 6, 2, 3, 0, 2, 3, 0, 1, 1, 5, 6, 3, 4, 0, 1, 1, 5],[5,1,3,6,4,2,7])
#t.applyReversals()
#t.array

In [54]:
def MSRSolve(n):
    bestFitness=float('inf')
    t=random.sample(range(1,n+1),n)
    start=copy.deepcopy(t)
    code=[]
    while bestFitness>20:
        msr=MSR(t)
        unit=msr.solve()
        bestFitness=unit.fitness
        t=unit.array
        code.append(unit.code)

    breakPoint = -1
    for i in range(1,len(start)):
        if abs(t[i-1]-t[i])>1:
            breakPoint = i

    #Popravi jedan breakpoint
    if breakPoint != -1:
        if abs(t[0] - t[breakPoint]) == 1: #levi
            for l in range(int((breakPoint)/2)):
                t[l],t[breakPoint-1-l] = t[breakPoint-1-l],t[l]
            code.append([0])
            code.append([breakPoint-1])

        elif abs(t[breakPoint-1] - t[-1]) == 1: #desni
            for l in range(int((len(t)-breakPoint)/2)):
                t[breakPoint+l],t[len(t)-1-l] = t[len(t)-1-l],t[breakPoint+l]
            code.append([breakPoint])
            code.append([len(t)-1])

        else: #abs(t[0] - t[-1]) == 1: #oba
            for l in range(int((breakPoint)/2)):
                t[l],t[breakPoint-1-l] = t[breakPoint-1-l],t[l]
            for l in range(int((len(t)-breakPoint)/2)):
                t[breakPoint+l],t[len(t)-1-l] = t[len(t)-1-l],t[breakPoint+l]
            code.append([0])
            code.append([breakPoint-1])
            code.append([breakPoint])
            code.append([len(t)-1])

    #Popravi ako je u opadajucem redosledu
    if t[0] > t[1]:
        for i in range(int(len(t)/2)):
            t[i],t[len(t)-1-i] = t[len(t)-1-i],t[i]
        code.append([0])
        code.append([len(t)-1])
    
    code2=[item for sublist in code for item in sublist]
    t = individual(code2,start)
    print("Done!")
    print("Code: " + str(t.code))
    print("Broj inverzija: " + str(len(t.code)/2))
    print("Start Array: " + str(start))
    print("End Array: " + str(t.array))
    print("Fitness: " + str(t.fitness))
    return t

In [57]:
avg = 0
numberOfIters = 5
numberRange = 10
for i in range(5,numberRange+1):
    for j in range(numberOfIters):
        t = MSRSolve(i)
        avg+=int(len(t.code)/2)
    avg/=numberOfIters
    with open("rez.txt","a") as f:
        f.write(str(len(t.array)) + " " + str(avg) + '\r\n')

Niz: [2, 1, 3, 4, 5, 6, 7]
Code: [2, 4]

Najbolja jedinka:
Fitness: 12
Niz: [2, 1, 3, 4, 5, 6, 7]
Code: [2, 4]

Najbolja jedinka:
Fitness: 12
Niz: [2, 1, 3, 4, 5, 6, 7]
Code: [2, 4]

Done!
Code: [1, 4, 2, 4, 4, 2, 5, 4, 4, 5, 2, 4, 0, 1]
Broj inverzija: 7.0
Start Array: [2, 5, 4, 6, 1, 3, 7]
End Array: [1, 2, 3, 4, 5, 6, 7]
Fitness: 14
Najbolja jedinka:
Fitness: 42
Niz: [6, 5, 2, 1, 7, 4, 3]
Code: [4, 5, 2, 3, 5, 6, 1, 2, 2, 3, 1, 2]

Najbolja jedinka:
Fitness: 16
Niz: [6, 5, 4, 3, 2, 1, 7]
Code: [1, 2, 5, 6, 2, 5]

Najbolja jedinka:
Fitness: 16
Niz: [6, 5, 4, 3, 2, 1, 7]
Code: [1, 2, 5, 6, 2, 5]

Najbolja jedinka:
Fitness: 16
Niz: [6, 5, 4, 3, 2, 1, 7]
Code: [1, 2, 5, 6, 2, 5]

Najbolja jedinka:
Fitness: 16
Niz: [6, 5, 4, 3, 2, 1, 7]
Code: [1, 2, 5, 6, 2, 5]

Done!
Code: [1, 2, 5, 6, 2, 5, 0, 5]
Broj inverzija: 4.0
Start Array: [6, 1, 5, 2, 3, 7, 4]
End Array: [1, 2, 3, 4, 5, 6, 7]
Fitness: 8
Najbolja jedinka:
Fitness: 36
Niz: [4, 5, 6, 7, 3, 1, 2]
Code: [3, 4, 4, 5, 2, 3, 4, 5, 4, 5,